In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.datasets import load_iris

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

import os
os.getcwd()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import tqdm
import time

In [5]:
df = pd.read_csv("/home/saraiva/Documentos/Dataframes/travel-insurance/travel_insurance.csv")
df = load_iris()

X = pd.DataFrame(data = df.data,columns = df.feature_names)
y = pd.DataFrame(df.target)

In [6]:
param_grid  = {'bootstrap' :[True,False], 
               'max_depth' : list(range(2, 10)),
               'max_features' : ['auto','sqrt','log2'],
               'min_samples_leaf': list(range(5,20)), 
               'n_estimators': list(range(30, 300 ,5))}

In [7]:
algo = RandomForestClassifier
metric_list = [accuracy_score]

In [11]:
def random_objective(params,
                     algo,
                     metric_list,
                     X, y,
                     iterations = 100):
    """Random search objective function. Takes in hyperparameters
       and returns a list of results to be saved."""
    
    #Define as colunas do dataframe de saida
    param_col = [key for key in param_grid.keys()] + [metrics.__name__ for metrics in metric_list]
    
    lots_vals = list()
    for rounds in tqdm.tqdm_notebook(range(iterations)):
        #Aleatoriza os parametros
        params = { key : random.sample(value,1)[0] for key,value in param_grid.items()}

        # Perform n_folds cross validation
        kfolds = KFold(n_splits=10,shuffle=True)

        matrix_metrics = []
        for train_idx,test_idx in kfolds.split(X):

            X_train,X_test = X.loc[train_idx,:],X.loc[test_idx,:]
            y_train,y_test = y.loc[train_idx,:],y.loc[test_idx,:]
            
            new_algo = algo(**params)

            new_algo.fit(X_train,np.ravel(y_train))
            y_pred = new_algo.predict(X_test)


            evaluation_list = []
            for evaluations in metric_list:
                result = evaluations(y_test,y_pred)
                evaluation_list.append(result)
            matrix_metrics.append(evaluation_list)

        #Dataframe criado com os resultados das metricas em cada fold
        aux_metrics = pd.DataFrame(matrix_metrics,columns = [metrics.__name__ for metrics in metric_list])
        metrics_results = aux_metrics.mean().tolist()


        # Return list of results
        vals = list(params.values()) + metrics_results
    
        lots_vals.append(vals)
    
    return pd.DataFrame(lots_vals,columns = param_col)

In [12]:
random_objective(param_grid,algo,metric_list,X,y)

,bootstrap,max_depth,max_features,min_samples_leaf,n_estimators,accuracy_score
0,True,2,sqrt,17,75,0.940
1,False,9,auto,10,270,0.940
2,False,8,log2,11,185,0.940
3,False,6,sqrt,10,195,0.947
4,True,7,auto,7,100,0.933
5,True,7,log2,6,185,0.953
6,True,5,log2,7,275,0.953
7,True,2,auto,17,240,0.940
8,True,8,auto,8,45,0.947
9,True,9,log2,7,80,0.953
